In [4]:
import emcee
import numpy as np
import matplotlib.pyplot as plt
import os, sys

sys.path.append('/home/joeadamo/Research') #<- parent directory of dark emulator code
from DarkEmuPowerRSD import pkmu_nn, pkmu_hod
from CovA-NN-Emulator import CovNet

SyntaxError: invalid syntax (3524451259.py, line 8)